# Select Your IPTS 

In [ ]:
from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_display_counts_of_region_vs_stack.ui')

from __code.display_counts_of_region_vs_stack import ImageWindow, DisplayCountsVsStack

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Description 

Load a stack of images, select a region and then the average counts of the region vs the stack will be displayed

# Set up notebook

In [ ]:
%gui qt

# Select stack folder

In [ ]:
o_display = DisplayCountsVsStack(working_dir=system.System.get_working_dir())
o_display.select_input_folder()

# Select region and display counts 

In [ ]:
_image = ImageWindow(display_counts_vs_stack=o_display)
_image.show()